In [1]:
from parafoil import CamberThicknessAirfoil
from parafoil.passages import TurboRowPassage, TurboStationPassage


mean_rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=-60,
    outlet_angle=-51,
    upper_thick_prop=[0.015, 0.05, 0.05, 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018,
    angle_units="deg"
)
mean_rotor_pasage = TurboStationPassage(
    airfoil=mean_rotor_airfoil,
    spacing_to_chord=1.0,
    leading_edge_gap_to_chord=4,
    trailing_edge_gap_to_chord=4,
    type="camber",
)
hub_rotor_airfoil = mean_rotor_airfoil.clone(inlet_angle=-50, outlet_angle=-25)
tip_rotor_airfoil = mean_rotor_airfoil.clone(inlet_angle=-67, outlet_angle=-63)

hub_rotor_passage = mean_rotor_pasage.clone(airfoil=hub_rotor_airfoil)
tip_rotor_passage = mean_rotor_pasage.clone(airfoil=tip_rotor_airfoil)
blade = TurboRowPassage([hub_rotor_passage, mean_rotor_pasage, mean_rotor_pasage], radii=[0.1131, 0.1697, 0.2262])

turbo_profile = blade.get_profile()

import cadquery as cq
cq.exporters.export(turbo_profile, "rotor67.step")

from jupyter_cadquery import show
show(turbo_profile)


Overwriting auto display for cadquery Workplane and Shape


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [3]:
from typing import List, Sequence, Union
from meshql import GeometryQL, Selection
import cadquery as cq
from jupyter_cadquery import show_object
from parafoil.boundary_conditions import InletTotalBoundaryCondition, OutletStaticPressureBoundaryCondition

interior_right_side = Selection(type="interior", filter=lambda edge: edge.Center().x>0)
interior_left_side = Selection(type="interior", filter=lambda edge: edge.Center().x<0)

profile_bbox = turbo_profile.val().BoundingBox()

# print(mean_rotor_pasage.leading_edge_gap)


# Ray cast selector
class DirectionSelector(cq.selectors.Selector):
    def __init__(self, parent: cq.Workplane, selector_str: str) -> None:
        super().__init__()
        self.parent= parent
        self.bbox = parent.val().BoundingBox()
        self.bbox_center = parent.val().Center()
        self.selector_str = selector_str
        self.bbox_profile = cq.Workplane("XY", origin=self.bbox_center).box(self.bbox.xlen*2, self.bbox.ylen*2, self.bbox.zlen*2)

    def filter(self, objectList: Sequence[cq.Face]) -> List[cq.Face]:
        filtered_faces = []

        selector_face = self.bbox_profile.faces(self.selector_str).val()

        for face in objectList:
            faces = selector_face.facesIntersectedByLine(face.Center(), face.normalAt(), direction="AlongAxis")
            if len(faces) and faces[0] == selector_face:
                filtered_faces.append(face)

        return filtered_faces

with GeometryQL.gmsh() as geo:
    new_geo = (
        geo
        .load(
            turbo_profile,
            on_split= lambda split: (
                split
                .from_plane(base_pnt=(mean_rotor_pasage.leading_edge_gap*0.2, 0), angle=(90, 0, 0))
                .from_plane(base_pnt=(-mean_rotor_pasage.trailing_edge_gap*0.2, 0), angle=(90, 0, 0))
                .group(
                     lambda split: (
                        split
                        .from_ratios(
                            start=Selection(">Z", type="interior"), 
                            end=Selection("<Z", type="interior"),
                            ratios=[0.45, 0.95],
                            dir="away",
                            snap=True,
                        )
                        .from_normals(interior_left_side, axis=[(-1, 0, 0), (0, 1, 0)])
                        .from_normals(interior_right_side, axis=[(1, 0, 0), (0.5, -1, 0)])
                     )
                )
            ),
            max_dim=10,
            # use_cache=True
        )
        .faces(">X")
        .addBoundaryCondition(
           lambda i, face:  InletTotalBoundaryCondition(
                label="inlet",
                total_temperature=300,
                total_pressure=101325,
                flow_direction=(-face.normalAt()).toTuple(),
            )
        )
        .end()

        .faces("<X")
        .addBoundaryCondition(
            OutletStaticPressureBoundaryCondition(
                label="outlet",
                static_pressure=101325,
            )
        )
        .end()

        .faces(DirectionSelector(turbo_profile, ">Y"), type="exterior")
        .show()
        .end()

        .faces(DirectionSelector(turbo_profile, "<Y"), type="exterior")
        .show()
        .end()


        # .setTransfiniteAuto(max_nodes=200)
        # .generate(3)
        # .show("gmsh")
    )

# center = turbo_profile.val().Center()
# intersect_line = cq.Edge.makeLine(center, center + cq.Vector(0,1,0))
# show(geo.workplane, intersect_line)

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.07s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.03s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…